In [ ]:
import pandas as pd
import numpy as np

## Load data 

In [ ]:
#Artwork metadata

df_artworks = pd.read_csv('/root/work/datasets/train_mayors_style_encoded_with_url.csv')
df_artworks.head()

**Get unique artists**

In [ ]:
df_artists = df_artworks['artist'].drop_duplicates().to_frame()
df_artists.head()

In [ ]:
df_artists['artist resquest'] = df_artists['artist'].str.lower()
df_artists['artist resquest'] = df_artists['artist resquest'].str.replace(' ', '-')
df_artists.head()

## Fetch data from Wikiart API

In [ ]:
import requests
import json

In [ ]:
#Data to access to Wikiart API

api_access_key= '37183c6a47324dbb'
api_secret_key= '8f6f7c48b8177602'

headers = {'X-ApiKeys' : 'accessCode=' + api_access_key + '; secretCode=' + api_secret_key}

In [ ]:
df_fetch_data = pd.DataFrame(columns=['artistContentId', 'artistName', 
                                      'title', 'completitionYear', 'width', 'height', 'image',
                                      'contentId', 'yearAsString'])

In [ ]:
def fetch_data(artist_name):
    global df_fetch_data
    url = 'http://www.wikiart.org/en/App/Painting/PaintingsByArtist?artistUrl='+artist_name+'&json=2'
    r = requests.get(url, headers=headers)
    df = pd.DataFrame(r.json())
    df_fetch_data = pd.concat([df_fetch_data, df], ignore_index=True)
    return len(df_fetch_data)

In [ ]:
import math

frame = 200
max_complete_frames = math.floor(len(df_artists)/frame)
max_complete_frames

In [ ]:
import time

for i in np.arange(0, max_complete_frames):
    df_test =df_artists[i*frame : (i+1)*frame]
    df_test['artist resquest'].apply(fetch_data)
    df_fetch_data.to_csv('data_fetched.csv', index=False)
    print("Iteration " + str(i) )
    time.sleep(3601)

In [ ]:
if len(df_artists) % frame != 0:
    df_test = df_artists[max_complete_frames*frame : len(df_artists) ]
    df_test['artist resquest'].apply(fetch_data)
    df_fetch_data.to_csv('data_fetched.csv', index=False)

In [ ]:
df_fetch_data.to_csv('/root/work/datasets/url_fetched.csv', index=False)

### Join data

In [ ]:
artist_problems = []

In [ ]:
def getUrl(X):
    if (len(df_fetch_data[(df_fetch_data['artistName'] == X['artist']) & (df_fetch_data['title'] == X['title'])]) != 0):
        return df_fetch_data[(df_fetch_data['artistName'] == X['artist']) & (df_fetch_data['title'] == X['title'])]['image'].values[0]
    else:
        artist_problems.append({'artist': X['artist'], 'title': X['title']})

In [ ]:
df_artworks.head()

In [ ]:
df_artworks['imageUrl'] = df_artworks.apply(getUrl, axis=1)
df_artworks.head()

In [ ]:
df_artworks.shape

**Save data**

In [ ]:
df_artworks.to_csv('/root/work/datasets/train_mayors_style_encoded_with_url.csv', index=False)

In [ ]:
df_artworks[df_artworks['artist']== 'August Macke']

## Analize artist bugs

In [ ]:
df_artists_errors = pd.DataFrame(artist_problems)
df_artists_errors.head()

In [ ]:
df_artists_errors.to_csv('artist_to_check.csv', index=False)

In [ ]:
len(df_artists_errors['artist'].unique())

In [ ]:
df_fetch_data[df_fetch_data['artistName'] == 'Ernst Ludwig Kirchner']